# Code Package Demo

A code package is a way of organizing a set of dependent files in a folder structure. 

Each code package has a main executable file that in turn accesses other files in the code package. For example, a code package can contain a notebook file that calls other notebook files or functions in scripts in the folder structure in a notebook job. See [docs](https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=integration-code-packages) for more details.

Currently, you can use code packages in deployment spaces only. 

So this notebook shows you an example of how to deploy code packages to Deployment Space using cpdctl tool.

# Table of Contents

1. [Install the version v1.1.132 of `cpdctl`](#1)
2. [Prepare code package](#2)
3. [Upload code package zip file to target space](#3)
4. [Create a code package asset](#4)
5. [Create a job](#5)
6. [Kick off the job](#6)
7. [Check the job log](#7)

<div class="alert alert-block alert-danger">
    

1. The code package includes 1 notebook and 3 python scripts: <br>
Code_pkg_notebook.ipynb calls data_prep.py for data preparation, then model_build.py for building models, and finally model_deploy.py for model deployment.

2. The Code_pkg_demo.ipynb prepares and deploys this code package to deployment space using cpdctl, and create a notebook job for the code_pkg_notebook.ipynb which requires 3 environment variables: <br>
    
    
    *space_name=Customer Attrition Space*  
    *model_name=test model*  
    *deployment_name=test model deployment*

Note: make sure you have promoted [customer_history.csv](./../assets/data_asset/customer_history.csv) to your deployment space.
    

   
</div>

![](./../imgs/code_pkg_demo_workflow.png)

In [1]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from IPython.core.display import display, HTML

<a id="1"></a>
## 1. Install the version v1.1.132 of `cpdctl`

In [2]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')
version='v1.1.132'

response = requests.get(CPDCTL_RELEASES_URL)
asset_version = next(a for a in response.json() if version==a['tag_name'])
#assets = response.json()[0]['assets']
assets=asset_version['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [4]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.1.132


<a id="2"></a>
## 2. Prepare code package

In [5]:
# prepare code pkg zip file
import shutil
shutil.make_archive("code_pkg", 'zip', "code_pkg")

'/userfs/assets/notebooks/code_pkg.zip'

In [6]:
remote_file_path = "code_package/cpdctl-test-code-package.zip"
local_file_path = "code_pkg.zip"

<a id="3"></a>
## 3. Upload code package zip file to target space

In [7]:
space_id = "0eaef2ee-dd8a-4292-8c2d-3edcdc0da0d0"

In [8]:
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --space-id {space_id}

...
OK


<a id="4"></a>
## 4. Create a code package asset.

In [9]:
os.environ["CPDCTL_ENABLE_CODE_PACKAGE"] = "true"

In [10]:
file_name = "code-package-demo.zip"

In [11]:
# get code_package_id

result = ! cpdctl code-package create --file-reference {remote_file_path} --name {file_name} --space-id {space_id} --output json -j "metadata.asset_id" --raw-output
code_package_id = result.s

# or run this to grab code_pkg_id ! cpdctl code-package create --file-reference {remote_file_path} --name {file_name} --space-id {space_id} --output json -j "metadata.asset_id" --raw-output

#code_package_id = "4ff4ef99-b1f7-46e6-ab93-163bbc2ee3b3"
print("code package id: {}".format(code_package_id))

code package id: e2f1268f-1c25-40f0-986c-6350e5a67613


<a id="5"></a>
## 5. Create a job

In [18]:
# get environment id
# environment_name = "IBM Runtime 22.1 on Python 3.9"
environment_name = "Default Python 3.8"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

result = ! cpdctl environment list --space-id {space_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

environment id: jupconda38-0eaef2ee-dd8a-4292-8c2d-3edcdc0da0d0


In [21]:
job_name = "code-package-job-demo1"
job = {
    'asset_ref': code_package_id, 
    'configuration': {
        'env_id': env_id, 
        'env_variables': [
            'space_name=Customer%20Attrition%20Space', 
            'model_name=test%20model',
            'deployment_name=test%20model%20deployment'
        ],
        'entrypoint': "code_pkg_notebook.ipynb"
    }, 
    'description': 'my code package job', 
    'name': job_name
}
job_json = json.dumps(job)

In [22]:
# create the job

result = ! cpdctl job create --job '{job_json}' --space-id {space_id} --output json -j "metadata.asset_id" --raw-output
job_id = result.s
print("job id: {}".format(job_id))

job id: ae31963e-011e-4eb1-b2ee-8abd1da77bec


<a id="6"></a>
## 6. Kick off the job

In [23]:
# kick off the job

job_run = {
    'configuration': {
        'env_variables': [
          'space_name=Customer%20Attrition%20Space', 
            'model_name=test%20model',
            'deployment_name=test%20model%20deployment'
        ]
    }
}
job_run_json = json.dumps(job_run)

result = ! cpdctl job run create --space-id {space_id} --job-id {job_id} --job-run '{job_run_json}' --output json -j "metadata.asset_id" --raw-output
run_id = result.s
#! cpdctl job run create --space-id "5ff4a90a-43a3-4b71-9a4a-7c8b3a0c27e5" --job-id "e3b5f948-b39d-48c9-ae7e-f2972a573dbd" --job-run '{job_run_json}' --output json -j "metadata.asset_id" --raw-output

In [24]:
#run_id = ""
print("run id: {}".format(run_id))

run id: FAILED                      ID:            2c5172ed-7947-4039-9b2e-a467a0cd345f    Name:          Notebook Job    Created:       2022-05-03T14:50:20Z    Description:       State:         Failed    Tags:          []   


<a id="7"></a>
## 7. Check the job log

In [25]:
# check job log
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --space-id {space_id}

...
FAILED


              
Code:      400   
Message:   The server cannot or will not process the request due to an apparent client error (e.g. malformed request syntax).   
Error:     Bad Request   
Reason:    Invalid resource guid format 'runId'   
